# Explore New Incident File



### Contents

1. [A](#1) <br>
    1.1. [AA](#1.1) <br>
    1.2. [AAA](#1.2) <br>
    1.3. [AAAA](#1.3) <br>
2. [B](#2) <br>
    2.1. [BB](#2.1) <br>

In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# tensorflow
import tensorflow as tf
from tensorflow import keras

from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Dropout, Conv1D, RepeatVector, TimeDistributed
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor

# Common imports
import os
import timeit
import numpy as np
import pandas as pd
import seaborn as sns
from math import sqrt
from datetime import date
import holidays
sns.set()
import warnings
warnings.filterwarnings("ignore")

# To plot pretty figures
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["font.sans-serif"] = "Verdana"
mpl.rcParams["lines.markersize"] = 20
# mpl.rc('axes', labelsize=14)
# mpl.rc('xtick', labelsize=12)
# mpl.rc('ytick', labelsize=12)

## VDS Lat Long.csv

In [4]:
loc_df = pd.read_csv("data/VDS Lat Long.csv", header=0)

In [17]:
# Extract only Kwinana Northbound (inner) main road
loc_df = loc_df[(loc_df['Short Name'].str.contains("KWN-M")) &
      (loc_df['Latitude'] >= -32.091154) & 
      (loc_df['Latitude'] <= -31.96255087)]

In [19]:
# Check minimum and maximum latitude
loc_df.describe()

,Latitude,Longitude
count,50.000000,50.000000
mean,-32.034439,115.852021
std,0.038955,0.003635
min,-32.091154,115.846104
25%,-32.068862,115.849048
50%,-32.042161,115.851549
75%,-32.008197,115.855041
max,-31.966753,115.860188


In [20]:
loc_df

,Short Name,Latitude,Longitude
6,0002KWN-MU,-31.969905,115.846725
7,0003KWN-MU,-31.988385,115.853238
201,0080KWN-MU,-31.966753,115.846104
203,0081KWN-MUL,-31.971349,115.847240
204,0081KWN-MUT,-31.971322,115.847235
211,0082KWN-MUL,-31.975453,115.847930
212,0082KWN-MUT,-31.975427,115.847925
217,0083KWN-MUL,-31.985870,115.852111
218,0083KWN-MUT,-31.985845,115.852099
223,0084KWN-MUL,-32.008217,115.855277


## Historic_Incidents_2016_-_2019__Points_.csv

In [23]:
inc_df=pd.read_csv("data/Historic_Incidents_2016_-_2019__Points_.csv", header=0)

In [33]:
(inc_df.Lat == inc_df.Y).value_counts()

True     24693
False     7784
dtype: int64

In [35]:
(inc_df.Long == inc_df.X).value_counts()

True    32477
dtype: int64

In [37]:
df = inc_df[['Lat', 'Long', 'WST_Start', 'WST_End', 'Incident_Type',
       'Description', 'Road', 'Location', 'TrafficCondition']]; df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32477 entries, 0 to 32476
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Lat               32477 non-null  float64
 1   Long              32477 non-null  float64
 2   WST_Start         32477 non-null  object 
 3   WST_End           32469 non-null  object 
 4   Incident_Type     32477 non-null  object 
 5   Description       32475 non-null  object 
 6   Road              29090 non-null  object 
 7   Location          32461 non-null  object 
 8   TrafficCondition  29090 non-null  object 
dtypes: float64(2), object(7)
memory usage: 2.2+ MB


In [46]:
df.WST_Start = pd.to_datetime(df.WST_Start)
df.WST_End = pd.to_datetime(df.WST_End)

In [118]:
# Extract only study period
df2 = df[(df.Road == 'Kwinana Fwy') &
  (df.Lat >= -32.091154) &                  # 0080KWN-MU
  (df.Lat <= -31.96323) &                   # Narrow Bridge
  (df.WST_Start >= '2018-01-01 00:00:00') & # between 1 Jan 2018
  (df.WST_End <= '2018-10-25 23:59:00') &   # to 25 Oct 2018
  (df.Description.str.contains('Kwinana Fwy northbound') |
  df.Description.str.contains('Kwinana Fwy both directions'))]; df2

,Lat,Long,WST_Start,WST_End,Incident_Type,Description,Road,Location,TrafficCondition
11401,-31.967082,115.846103,2018-01-01 16:21:00,2018-01-01 16:59:00,Break Down / Tow Away,Kwinana Fwy northbound Narrows Bridge South Pe...,Kwinana Fwy,Right Lane(s) Blocked. Congestion. Exercise ...,Right Lane(s) Blocked
11442,-32.033494,115.858312,2018-01-03 15:40:59,2018-01-03 15:56:00,Break Down / Tow Away,Kwinana Fwy northbound Mt Henry Bridge Salter ...,Kwinana Fwy,Right Emergency Lane Blocked. No Known Conges...,Right Emergency Lane Blocked
11474,-32.025228,115.859856,2018-01-04 17:10:00,2018-01-04 19:20:00,Break Down / Tow Away,Kwinana Fwy northbound after Mt Henry Bridge S...,Kwinana Fwy,Left Emergency Lane Blocked. No Known Congest...,Left Emergency Lane Blocked
11486,-32.043923,115.854653,2018-01-05 05:19:00,2018-01-05 05:58:00,Road Crash,Kwinana Fwy northbound at Leach Hwy Brentwood ...,Kwinana Fwy,Centre Lane(s) Blocked. Congestion. Exercise...,Centre Lane(s) Blocked
11553,-32.011120,115.856484,2018-01-09 09:10:00,2018-01-09 09:40:00,Debris / Trees / Lost Loads,Kwinana Fwy northbound Canning Hwy Applecross ...,Kwinana Fwy,Centre Lane(s) Blocked. No Known Congestion. ...,Centre Lane(s) Blocked
...,...,...,...,...,...,...,...,...,...
18634,-31.984470,115.851317,2018-10-22 08:25:00,2018-10-22 09:42:00,Debris / Trees / Lost Loads,Kwinana Fwy northbound prior to South Tce Sout...,Kwinana Fwy,All Lanes Open. No Known Congestion. Exercis...,All Lanes Open
18635,-31.993695,115.855077,2018-10-22 08:25:00,2018-10-22 09:42:00,Debris / Trees / Lost Loads,Kwinana Fwy northbound prior to South Tce Sout...,Kwinana Fwy,All Lanes Open. No Known Congestion. Exercis...,All Lanes Open
18636,-32.082478,115.848946,2018-10-22 08:32:59,2018-10-22 10:44:59,Debris / Trees / Lost Loads,Kwinana Fwy northbound after Roe Hwy Bibra Lak...,Kwinana Fwy,Lane Closures Unknown. Congestion. Exercise ...,Lane Closures Unknown
18670,-31.963448,115.846710,2018-10-23 15:49:59,2018-10-23 16:02:00,Road Crash,Kwinana Fwy northbound on the Narrows Bridge -...,Kwinana Fwy,Right Centre Lane(s) Blocked. Heavy Congestio...,Right Centre Lane(s) Blocked


In [119]:
df2[['Condition', 'Congestion', 'Caution']] = df2['Location'].str.split('[.]\s\s',2,expand=True)

In [120]:
df2

,Lat,Long,WST_Start,WST_End,Incident_Type,Description,Road,Location,TrafficCondition,Condition,Congestion,Caution
11401,-31.967082,115.846103,2018-01-01 16:21:00,2018-01-01 16:59:00,Break Down / Tow Away,Kwinana Fwy northbound Narrows Bridge South Pe...,Kwinana Fwy,Right Lane(s) Blocked. Congestion. Exercise ...,Right Lane(s) Blocked,Right Lane(s) Blocked,Congestion,Exercise Extreme Caution.
11442,-32.033494,115.858312,2018-01-03 15:40:59,2018-01-03 15:56:00,Break Down / Tow Away,Kwinana Fwy northbound Mt Henry Bridge Salter ...,Kwinana Fwy,Right Emergency Lane Blocked. No Known Conges...,Right Emergency Lane Blocked,Right Emergency Lane Blocked,No Known Congestion,Exercise Extreme Caution.
11474,-32.025228,115.859856,2018-01-04 17:10:00,2018-01-04 19:20:00,Break Down / Tow Away,Kwinana Fwy northbound after Mt Henry Bridge S...,Kwinana Fwy,Left Emergency Lane Blocked. No Known Congest...,Left Emergency Lane Blocked,Left Emergency Lane Blocked,No Known Congestion,Exercise Extreme Caution.
11486,-32.043923,115.854653,2018-01-05 05:19:00,2018-01-05 05:58:00,Road Crash,Kwinana Fwy northbound at Leach Hwy Brentwood ...,Kwinana Fwy,Centre Lane(s) Blocked. Congestion. Exercise...,Centre Lane(s) Blocked,Centre Lane(s) Blocked,Congestion,Exercise Extreme Caution.
11553,-32.011120,115.856484,2018-01-09 09:10:00,2018-01-09 09:40:00,Debris / Trees / Lost Loads,Kwinana Fwy northbound Canning Hwy Applecross ...,Kwinana Fwy,Centre Lane(s) Blocked. No Known Congestion. ...,Centre Lane(s) Blocked,Centre Lane(s) Blocked,No Known Congestion,Exercise Extreme Caution.
...,...,...,...,...,...,...,...,...,...,...,...,...
18634,-31.984470,115.851317,2018-10-22 08:25:00,2018-10-22 09:42:00,Debris / Trees / Lost Loads,Kwinana Fwy northbound prior to South Tce Sout...,Kwinana Fwy,All Lanes Open. No Known Congestion. Exercis...,All Lanes Open,All Lanes Open,No Known Congestion,Exercise Extreme Caution.
18635,-31.993695,115.855077,2018-10-22 08:25:00,2018-10-22 09:42:00,Debris / Trees / Lost Loads,Kwinana Fwy northbound prior to South Tce Sout...,Kwinana Fwy,All Lanes Open. No Known Congestion. Exercis...,All Lanes Open,All Lanes Open,No Known Congestion,Exercise Extreme Caution.
18636,-32.082478,115.848946,2018-10-22 08:32:59,2018-10-22 10:44:59,Debris / Trees / Lost Loads,Kwinana Fwy northbound after Roe Hwy Bibra Lak...,Kwinana Fwy,Lane Closures Unknown. Congestion. Exercise ...,Lane Closures Unknown,Lane Closures Unknown,Congestion,Exercise Extreme Caution.
18670,-31.963448,115.846710,2018-10-23 15:49:59,2018-10-23 16:02:00,Road Crash,Kwinana Fwy northbound on the Narrows Bridge -...,Kwinana Fwy,Right Centre Lane(s) Blocked. Heavy Congestio...,Right Centre Lane(s) Blocked,Right Centre Lane(s) Blocked,Heavy Congestion,Delays Expected.


In [121]:
# interchangable
df2[(df2.TrafficCondition != df2.Condition)]

,Lat,Long,WST_Start,WST_End,Incident_Type,Description,Road,Location,TrafficCondition,Condition,Congestion,Caution
12425,-32.030686,115.859977,2018-02-15 08:51:00,2018-02-15 10:11:00,Debris / Trees / Lost Loads,Kwinana Fwy northbound after Mount Henry Bridg...,Kwinana Fwy,All Lanes Open (left and centre lanes affected...,All Lanes Open,All Lanes Open (left and centre lanes affected),Congestion,Exercise Extreme Caution.
15969,-32.007637,115.854945,2018-07-13 07:54:00,2018-07-13 07:59:00,Road Crash,Kwinana Fwy northbound Canning Hwy Como - Roa...,Kwinana Fwy,Left Lane Blocked. Congestion. Exercise Extr...,Left Lane(s) Blocked,Left Lane Blocked,Congestion,Exercise Extreme Caution.
16073,-32.021402,115.857566,2018-07-18 12:51:00,2018-07-18 14:23:59,Road Crash,Kwinana Fwy northbound prior to Canning Hwy Sa...,Kwinana Fwy,Left Lane Blocked. Congestion. Exercise Extr...,Left Lane(s) Blocked,Left Lane Blocked,Congestion,Exercise Extreme Caution.


In [122]:
df2.TrafficCondition.value_counts()

Left Emergency Lane Blocked     131
All Lanes Open                   89
Left Lane(s) Blocked             55
Right Lane(s) Blocked            45
Lane Closures Unknown            22
Right Emergency Lane Blocked     19
Centre Lane(s) Blocked           14
Left Centre Lane(s) Blocked       8
Right Centre Lane(s) Blocked      3
Left Turning Pocket Blocked       3
Right Turning Pocket Blocked      1
Bus Lane Blocked                  1
Name: TrafficCondition, dtype: int64

In [123]:
df2.Congestion.value_counts()

No Known Congestion    210
Congestion             125
Heavy Congestion        56
Name: Congestion, dtype: int64

In [124]:
df2.Caution.value_counts()

Exercise Extreme Caution.       324
Delays Expected.                 43
Allow Extra Travel Time.         18
Emergency Services Onsite.        4
Incident Cleared.                 1
Traffic Management In Place.      1
Name: Caution, dtype: int64

In [125]:
df2 = df2[['Lat', 'Long', 'WST_Start', 'WST_End',
          'Incident_Type', 'TrafficCondition',
           'Congestion', 'Caution']]; df2

,Lat,Long,WST_Start,WST_End,Incident_Type,TrafficCondition,Congestion,Caution
11401,-31.967082,115.846103,2018-01-01 16:21:00,2018-01-01 16:59:00,Break Down / Tow Away,Right Lane(s) Blocked,Congestion,Exercise Extreme Caution.
11442,-32.033494,115.858312,2018-01-03 15:40:59,2018-01-03 15:56:00,Break Down / Tow Away,Right Emergency Lane Blocked,No Known Congestion,Exercise Extreme Caution.
11474,-32.025228,115.859856,2018-01-04 17:10:00,2018-01-04 19:20:00,Break Down / Tow Away,Left Emergency Lane Blocked,No Known Congestion,Exercise Extreme Caution.
11486,-32.043923,115.854653,2018-01-05 05:19:00,2018-01-05 05:58:00,Road Crash,Centre Lane(s) Blocked,Congestion,Exercise Extreme Caution.
11553,-32.011120,115.856484,2018-01-09 09:10:00,2018-01-09 09:40:00,Debris / Trees / Lost Loads,Centre Lane(s) Blocked,No Known Congestion,Exercise Extreme Caution.
...,...,...,...,...,...,...,...,...
18634,-31.984470,115.851317,2018-10-22 08:25:00,2018-10-22 09:42:00,Debris / Trees / Lost Loads,All Lanes Open,No Known Congestion,Exercise Extreme Caution.
18635,-31.993695,115.855077,2018-10-22 08:25:00,2018-10-22 09:42:00,Debris / Trees / Lost Loads,All Lanes Open,No Known Congestion,Exercise Extreme Caution.
18636,-32.082478,115.848946,2018-10-22 08:32:59,2018-10-22 10:44:59,Debris / Trees / Lost Loads,Lane Closures Unknown,Congestion,Exercise Extreme Caution.
18670,-31.963448,115.846710,2018-10-23 15:49:59,2018-10-23 16:02:00,Road Crash,Right Centre Lane(s) Blocked,Heavy Congestion,Delays Expected.


In [126]:
# Duplicate entries
df2.WST_Start.round('min').value_counts()

2018-07-25 09:41:00    3
2018-05-29 14:43:00    2
2018-05-22 06:58:00    2
2018-07-14 10:55:00    2
2018-04-15 10:23:00    2
                      ..
2018-04-09 15:04:00    1
2018-04-09 13:17:00    1
2018-04-06 18:13:00    1
2018-04-06 17:32:00    1
2018-10-25 08:51:00    1
Name: WST_Start, Length: 372, dtype: int64

In [127]:
# Drop duplicates
df3 = df2.drop_duplicates(subset=['WST_Start', 'WST_End', 
                            'Incident_Type', 'TrafficCondition',
                           'Congestion', 'Caution'])
df3.WST_Start.round('min').value_counts()

2018-01-01 16:21:00    1
2018-07-20 09:39:00    1
2018-07-25 12:38:00    1
2018-07-25 12:00:00    1
2018-07-25 09:41:00    1
                      ..
2018-04-09 15:04:00    1
2018-04-09 13:17:00    1
2018-04-06 18:13:00    1
2018-04-06 17:32:00    1
2018-10-25 08:51:00    1
Name: WST_Start, Length: 372, dtype: int64

In [128]:
df3

,Lat,Long,WST_Start,WST_End,Incident_Type,TrafficCondition,Congestion,Caution
11401,-31.967082,115.846103,2018-01-01 16:21:00,2018-01-01 16:59:00,Break Down / Tow Away,Right Lane(s) Blocked,Congestion,Exercise Extreme Caution.
11442,-32.033494,115.858312,2018-01-03 15:40:59,2018-01-03 15:56:00,Break Down / Tow Away,Right Emergency Lane Blocked,No Known Congestion,Exercise Extreme Caution.
11474,-32.025228,115.859856,2018-01-04 17:10:00,2018-01-04 19:20:00,Break Down / Tow Away,Left Emergency Lane Blocked,No Known Congestion,Exercise Extreme Caution.
11486,-32.043923,115.854653,2018-01-05 05:19:00,2018-01-05 05:58:00,Road Crash,Centre Lane(s) Blocked,Congestion,Exercise Extreme Caution.
11553,-32.011120,115.856484,2018-01-09 09:10:00,2018-01-09 09:40:00,Debris / Trees / Lost Loads,Centre Lane(s) Blocked,No Known Congestion,Exercise Extreme Caution.
...,...,...,...,...,...,...,...,...
18621,-31.997337,115.854986,2018-10-21 16:57:00,2018-10-21 17:25:00,Road Crash,Left Lane(s) Blocked,Congestion,Exercise Extreme Caution.
18634,-31.984470,115.851317,2018-10-22 08:25:00,2018-10-22 09:42:00,Debris / Trees / Lost Loads,All Lanes Open,No Known Congestion,Exercise Extreme Caution.
18636,-32.082478,115.848946,2018-10-22 08:32:59,2018-10-22 10:44:59,Debris / Trees / Lost Loads,Lane Closures Unknown,Congestion,Exercise Extreme Caution.
18670,-31.963448,115.846710,2018-10-23 15:49:59,2018-10-23 16:02:00,Road Crash,Right Centre Lane(s) Blocked,Heavy Congestion,Delays Expected.
